모델을 학습하는 과정은 반복적인 과정(에폭(epoch)이라고 부르는)  
각 반복 단계에서 모델은 출력을 추측하고,  
추측과 정답 사이의 오류(손실(loss))를 계산하고,  
매개변수에 대한 오류의 도함수(derivative)를 수집한 뒤,  
경사하강법을 사용하여 이 파라매터들을 최적화(optimize)

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

### 하이퍼파라매터(Hyperparameter)  
하이퍼파라매터(Hyperparameter)는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수  
서로 다른 하이퍼파라매터 값은 모델 학습과 수렴율(convergence rate)에 영향  
<br>
학습 시에는 다음과 같은 하이퍼파라매터를 정의합니다:
- 에폭(epoch) 수 - 데이터셋을 반복하는 횟수
- 배치 크기(batch size) - 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
- 학습률(learning rate) - 각 배치/에폭에서 모델의 매개변수를 조절하는 비율. 
> 값이 작을수록 학습 속도가 느려지고,  
> 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있습니다.

In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### 최적화 단계(Optimization Loop)
하이퍼파라매터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화  
> 에폭(epoch): 최적화 단계의 각 반복(iteration)
<br>
>  > 하나의 에폭은 다음 두 부분으로 구성
- 학습 단계(train loop) - 학습용 데이터셋을 반복(iterate)하고 최적의 매개변수로 수렴
- 검증/테스트 단계(validation/test loop) - 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복(iterate)

### 손실 함수(loss function)
손실 함수(loss function)는 획득한 결과와 실제 값 사이의 틀린 정도(degree of dissimilarity)를 측정(학습 중에 이 값을 최소화)  
<br>
일반적인 손실함수에는 회귀 문제(regression task)에 사용하는 <b>nn.MSELoss(평균 제곱 오차(MSE; Mean Square Error))</b>,  
분류(classification)에 사용하는 <b>nn.NLLLoss (음의 로그 우도(Negative Log Likelihood))</b>,  
그리고 nn.LogSoftmax와 nn.NLLLoss를 합친 <b>nn.CrossEntropyLoss</b> 등이 존재

In [7]:
# 학습하려는 모델의 매개변수와 학습률(learning rate) 하이퍼파라매터를 등록하여 옵티마이저를 초기화
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

학습 단계(loop)에서 최적화는 세단계
- optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정 
> 기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정
- loss.backwards()를 호출하여 예측 손실(prediction loss)을 역전파
> PyTorch는 각 매개변수에 대한 손실의 변화도를 저장
- 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.294652  [    0/60000]
loss: 2.284837  [ 6400/60000]
loss: 2.271302  [12800/60000]
loss: 2.280902  [19200/60000]
loss: 2.265997  [25600/60000]
loss: 2.243392  [32000/60000]
loss: 2.251141  [38400/60000]
loss: 2.220715  [44800/60000]
loss: 2.212228  [51200/60000]
loss: 2.208024  [57600/60000]
Test Error: 
 Accuracy: 45.6%, Avg loss: 2.189899 

Epoch 2
-------------------------------
loss: 2.187868  [    0/60000]
loss: 2.178248  [ 6400/60000]
loss: 2.134462  [12800/60000]
loss: 2.161128  [19200/60000]
loss: 2.116519  [25600/60000]
loss: 2.071243  [32000/60000]
loss: 2.094137  [38400/60000]
loss: 2.028884  [44800/60000]
loss: 2.025100  [51200/60000]
loss: 1.982394  [57600/60000]
Test Error: 
 Accuracy: 58.7%, Avg loss: 1.965783 

Epoch 3
-------------------------------
loss: 1.982036  [    0/60000]
loss: 1.955594  [ 6400/60000]
loss: 1.857994  [12800/60000]
loss: 1.904712  [19200/60000]
loss: 1.801319  [25600/60000]
loss: 1.747926  [32000/600